In [4]:
import pandas as pd
import numpy as np

In [65]:
companies = pd.read_csv("companies.csv")
companies.shape

(66368, 10)

In [66]:
rounds2 = pd.read_csv("rounds2.csv")
rounds2.shape

(114949, 6)

# Checkpoint 1

In [67]:
companies.permalink = companies.permalink.str.upper()
rounds2.company_permalink = rounds2.company_permalink.str.upper()

In [68]:
distinct_companies = companies.permalink.unique()
distinct_companies_r = rounds2.company_permalink.unique()
print('Distinct companies in companies table: ',distinct_companies.shape)
print('Distinct companies in rounds2 table: ',distinct_companies_r.shape)

Distinct companies in companies table:  (66368,)
Distinct companies in rounds2 table:  (66368,)


In [69]:
z = companies.permalink.isin(rounds2.company_permalink)
print("Total companies present  from companies table in rounds2 table : ",z.count())

Total companies present  from companies table in rounds2 table :  66368


In [70]:
master_frame = rounds2.merge(companies, left_on ='company_permalink', right_on = 'permalink',how = 'outer')
master_frame.shape

(114949, 16)

# checkpoint 2

In [71]:
master_frame.funding_round_type = master_frame.funding_round_type.str.lower()
funding_type_list = ['venture','seed','angel','private_equity']
funding_type_df = master_frame[master_frame['funding_round_type'].isin(funding_type_list)]
funding_type_group = funding_type_df.groupby(['funding_round_type'])
funding_type_mean=funding_type_group[['raised_amount_usd']].mean().reset_index()

In [72]:
funding_type_mean = funding_type_mean.sort_values('raised_amount_usd',ascending=False)
funding_type_mean.query('raised_amount_usd >= 5000000 & raised_amount_usd <= 15000000')

,funding_round_type,raised_amount_usd
3,venture,1.174895e+07


# checkpoint 3

In [73]:
master_frame_venture = master_frame[master_frame['funding_round_type'] == 'venture']
top_9 = master_frame_venture.groupby(
                        'country_code').agg(
                                    {'raised_amount_usd':'sum'}).sort_values(
                                            'raised_amount_usd',ascending=False).reset_index().head(9)
top_9

,country_code,raised_amount_usd
0,USA,4.225108e+11
1,CHN,3.983542e+10
2,GBR,2.024563e+10
3,IND,1.439186e+10
4,CAN,9.583332e+09
5,FRA,7.259537e+09
6,ISR,6.907515e+09
7,DEU,6.346960e+09
8,JPN,3.363677e+09


# checkpoint 4

In [74]:
mapping = pd.read_csv("mapping.csv")
#print(mapping.shape)

mapping = mapping.dropna(thresh=10)
#print(mapping.head(6))

mapping.category_list = mapping.category_list.str.replace('0','na')
mapping.category_list = mapping.category_list.str.replace('2.na','2.0')
mapping = pd.melt(mapping,id_vars=['category_list'])
mapping = mapping[mapping['value'] == 1]
mapping = mapping.rename(index=str, columns={"variable": "main_sector"})
mapping['category_list'] = mapping['category_list'].str.lower()
del mapping['value']
print('data dimentions in mapping:',mapping.shape)
mapping.head(5)

data dimentions in mapping: (687, 2)


,category_list,main_sector
7,adventure travel,Automotive & Sports
13,aerospace,Automotive & Sports
44,auto,Automotive & Sports
45,automated kiosk,Automotive & Sports
46,automotive,Automotive & Sports


In [75]:
master_frame['primary_sector'] = master_frame.category_list.str.split('|',1).str[0]
master_frame['primary_sector'] = master_frame['primary_sector'].str.lower()
master_frame = master_frame.merge(mapping, left_on ='primary_sector', 
                                  right_on = 'category_list',how = 'left')
master_frame.head(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,primary_sector,category_list_y,main_sector
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,5/1/15,10000000.0,/ORGANIZATION/-FAME,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,media,media,Entertainment
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14,application platforms,application platforms,"News, Search and Messaging"
2,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,1/3/14,700000.0,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14,application platforms,application platforms,"News, Search and Messaging"
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,apps,apps,"News, Search and Messaging"
4,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/ORGANIZATION/0-6-COM,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,1/1/07,curated web,curated web,"News, Search and Messaging"


# Checkpoint 5

In [76]:
D1 = master_frame.query('funding_round_type == "venture" & country_code == "USA" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D1.shape

(12150, 19)

In [77]:
D2 = master_frame.query('funding_round_type == "venture" & country_code == "GBR" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D2.shape

(628, 19)

In [78]:
D3 = master_frame.query('funding_round_type == "venture" & country_code == "IND" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D3.shape

(330, 19)

In [79]:
def my_agg(x):
    names = {
        'investment_sector_count': x['raised_amount_usd'].count(),
        'investment_sector_total':  x['raised_amount_usd'].mean()}
    return pd.Series(names, index=['investment_sector_count', 'investment_sector_total'])

In [81]:
D1_group_df = D1.groupby('main_sector').apply(my_agg)
D1 = D1.merge(D1_group_df,left_on = 'main_sector', right_on = 'main_sector',how='left')

In [82]:
D2_group_df = D2.groupby('main_sector').apply(my_agg)
D2 = D2.merge(D1_group_df,left_on = 'main_sector', right_on = 'main_sector',how='left')

In [83]:
D3_group_df = D3.groupby('main_sector').apply(my_agg)
D3 = D3.merge(D1_group_df,left_on = 'main_sector', right_on = 'main_sector',how='left')

In [85]:
D1_top3 = D1.groupby('main_sector').agg(
                    {'raised_amount_usd':'count'}).sort_values(
                            'raised_amount_usd',ascending=False).reset_index().head(3)
D1_top3

,main_sector,raised_amount_usd
0,Others,2950
1,"Social, Finance, Analytics, Advertising",2714
2,Cleantech / Semiconductors,2350


In [87]:
D2_top3 = D2.groupby('main_sector').agg(
                {'raised_amount_usd':'count'}).sort_values(
                            'raised_amount_usd',ascending=False).reset_index().head(3)
D2_top3

,main_sector,raised_amount_usd
0,Others,147
1,"Social, Finance, Analytics, Advertising",133
2,Cleantech / Semiconductors,130


In [88]:
D3_top3 = D3.groupby('main_sector').agg(
                    {'raised_amount_usd':'count'}).sort_values(
                        'raised_amount_usd',ascending=False).reset_index().head(3)
D3_top3

,main_sector,raised_amount_usd
0,Others,110
1,"Social, Finance, Analytics, Advertising",60
2,"News, Search and Messaging",52


In [105]:
D1_top_companies = D1[D1.main_sector == D1_top3.main_sector[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in top sector:',D1_top_companies.name[:1])
D1_top_companies1 = D1[D1.main_sector == D1_top3.main_sector[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in second best sector:',D1_top_companies1.name[:1])

Top company in top sector: 0    Virtustream
Name: name, dtype: object
Top company in second best sector: 0    SST Inc. (Formerly ShotSpotter)
Name: name, dtype: object


In [106]:
D2_top_companies = D2[D2.main_sector == D2_top3.main_sector[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in top sector:',D2_top_companies.name[:1])
D2_top_companies1 = D2[D2.main_sector == D2_top3.main_sector[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in second best sector:',D2_top_companies1.name[:1])

Top company in top sector: 0    Electric Cloud
Name: name, dtype: object
Top company in second best sector: 0    Celltick Technologies
Name: name, dtype: object


In [107]:
D3_top_companies = D3[D3.main_sector == D3_top3.main_sector[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in top sector:',D3_top_companies.name[:1])
D3_top_companies1 = D3[D3.main_sector == D3_top3.main_sector[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False).reset_index()
print('Top company in second best sector:',D3_top_companies1.name[:1])

Top company in top sector: 0    FirstCry.com
Name: name, dtype: object
Top company in second best sector: 0    Manthan Systems
Name: name, dtype: object
